In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)
num_samples = 1000
num_features = 5
X = np.random.rand(num_samples, num_features)
true_coefficients = np.random.rand(num_features) * 10
noise = np.random.normal(loc=0, scale=1, size=num_samples)
y = np.dot(X, true_coefficients) + noise

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define a function to train and evaluate the neural network
def train_neural_network(hidden_layer_sizes, activation, solver):
    model = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    error = mean_squared_error(y_val, y_pred)
    return error

# Manually test different hyperparameters
best_error = float('inf')
best_params = None

# Try different configurations
hidden_layer_sizes_options = [(50,), (100,), (50, 50)]
activation_options = ['relu', 'tanh']
solver_options = ['adam', 'sgd']

for hidden_layer_sizes in hidden_layer_sizes_options:
    for activation in activation_options:
        for solver in solver_options:
            error = train_neural_network(hidden_layer_sizes, activation, solver)
            print(f"Tested {hidden_layer_sizes}, {activation}, {solver} => MSE: {error}")
            if error < best_error:
                best_error = error
                best_params = (hidden_layer_sizes, activation, solver)

print("\nBest parameters found:")
print(f"Hidden Layer Sizes: {best_params[0]}")
print(f"Activation: {best_params[1]}")
print(f"Solver: {best_params[2]}")
print(f"Validation Error (MSE): {best_error}")


Tested (50,), relu, adam => MSE: 3.5335765639845995
Tested (50,), relu, sgd => MSE: 1.038277552584494
Tested (50,), tanh, adam => MSE: 1.117501595654189
Tested (50,), tanh, sgd => MSE: 0.9794951097048008
Tested (100,), relu, adam => MSE: 3.462557072357153
Tested (100,), relu, sgd => MSE: 0.9966727195740722
Tested (100,), tanh, adam => MSE: 0.9316728673660731
Tested (100,), tanh, sgd => MSE: 0.9503517147628835
Tested (50, 50), relu, adam => MSE: 1.1009668508223676
Tested (50, 50), relu, sgd => MSE: 0.9389666217503263
Tested (50, 50), tanh, adam => MSE: 1.9042674149156453
Tested (50, 50), tanh, sgd => MSE: 0.9741193263887072

Best parameters found:
Hidden Layer Sizes: (100,)
Activation: tanh
Solver: adam
Validation Error (MSE): 0.9316728673660731


In [12]:
#MK optimal mse should be less than 4

import numpy as np, random
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)
X = (np.random.uniform(150, 200, (100, 3)) - 175) / 25
y = 0.3*X[:,0] - 0.2*X[:,1] + 0.1*X[:,2] + np.random.normal(0, 2, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

def create_nn(params):
    m = MLPRegressor(hidden_layer_sizes=(5,), max_iter=2, solver='lbfgs', warm_start=True)

    m.fit(X_train, y_train)
    p = np.array(params); i = 0
    for arr in m.coefs_ + m.intercepts_:
        s = np.prod(arr.shape)
        arr[...] = p[i:i+s].reshape(arr.shape)
        i += s
    return m

def fitness(p): return mean_squared_error(y_train, create_nn(p).predict(X_train))
def crossover(p1, p2): pt = random.randint(0, len(p1)-1); return p1[:pt]+p2[pt:]
def mutate(p, r=0.1): return [w+np.random.randn()*r if random.random()<0.1 else w for w in p]

tmp = MLPRegressor(hidden_layer_sizes=(5,), max_iter=1); tmp.fit(X_train, y_train)
n_params = sum(np.prod(a.shape) for a in tmp.coefs_ + tmp.intercepts_)
pop = [np.random.uniform(-1, 1, n_params).tolist() for _ in range(10)]

for g in range(10):
    pop = sorted(pop, key=fitness)
    new_pop = pop[:2]
    while len(new_pop) < len(pop):
        c = mutate(crossover(*random.sample(pop[:5], 2)))
        new_pop.append(c)
    pop = new_pop
    print(f"Gen {g+1}, MSE: {fitness(pop[0]):.4f}")

best = pop[0]
print("Test MSE:", mean_squared_error(y_test, create_nn(best).predict(X_test)))


Gen 1, MSE: 4.4953
Gen 2, MSE: 4.3959
Gen 3, MSE: 4.3556
Gen 4, MSE: 4.2915
Gen 5, MSE: 4.2915
Gen 6, MSE: 4.2766
Gen 7, MSE: 4.2766
Gen 8, MSE: 4.2393
Gen 9, MSE: 4.2308
Gen 10, MSE: 4.1749
Test MSE: 2.6147542088566462


In [13]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
import warnings
warnings.filterwarnings("ignore")
# Simulated spray drying dataset: [InletTemp, FeedRate, AtomSpeed] -> MoistureContent
np.random.seed(42)
X = np.random.uniform(low=150, high=200, size=(100, 3))  # features
y = 0.3*X[:,0] - 0.2*X[:,1] + 0.1*X[:,2] + np.random.normal(0, 2, 100)  # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Neural Network Model (MLP)
def create_nn(weights):
    model = MLPRegressor(hidden_layer_sizes=(5,), max_iter=1, warm_start=True)
    model.fit(X_train, y_train)
    weights = np.array(weights)  # <-- convert to NumPy array
    i = 0
    for layer_weights in model.coefs_:
        shape = layer_weights.shape
        model.coefs_[i] = weights[:np.prod(shape)].reshape(shape)
        weights = weights[np.prod(shape):]
        i += 1
    return model

# Genetic Algorithm
def fitness_function(weights):
    model = create_nn(weights)
    preds = model.predict(X_train)
    return mean_squared_error(y_train, preds)

def crossover(p1, p2):
    point = random.randint(0, len(p1)-1)
    return p1[:point] + p2[point:]

def mutate(ind, rate=0.1):
    return [w + np.random.randn()*rate if random.random() < 0.1 else w for w in ind]

# Initialize population
n_weights = (3*5) + (5*1)  # assuming 3 input, 5 hidden, 1 output
pop = [np.random.uniform(-1, 1, n_weights).tolist() for _ in range(10)]

# Run GA
for generation in range(10):
    pop = sorted(pop, key=fitness_function)
    new_pop = pop[:2]  # elitism
    while len(new_pop) < len(pop):
        p1, p2 = random.sample(pop[:5], 2)
        child = mutate(crossover(p1, p2))
        new_pop.append(child)
    pop = new_pop
    print(f"Gen {generation+1}, MSE: {fitness_function(pop[0]):.4f}")

# Final model
best_weights = pop[0]
final_model = create_nn(best_weights)
test_preds = final_model.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, test_preds))


Gen 1, MSE: 837.9926
Gen 2, MSE: 879.3118
Gen 3, MSE: 899.3248
Gen 4, MSE: 213.4704
Gen 5, MSE: 155.7211
Gen 6, MSE: 100.6344
Gen 7, MSE: 83.3831
Gen 8, MSE: 86.6404
Gen 9, MSE: 88.8074
Gen 10, MSE: 76.7427
Test MSE: 60.076637048881345
